In [1]:
print("Hello World")

Hello World


In [2]:
!pip3 install shutil os

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import shutil
import os

# 1. Define source and destination paths
src_path = "/Users/noel_personal/.cache/kagglehub/datasets/shenhuichang/badminton-storke-video/versions/4"
dst_path = "/Users/noel_personal/Desktop/badminton_dataset"

# 2. Move the dataset (or copy it)
try:
    shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
    print(f"✅ Dataset successfully copied to:\n{dst_path}")
except Exception as e:
    print(f"❌ Error during copy: {e}")

# 3. List all video files for verification
print("\n📂 Video files found:\n")
for root, dirs, files in os.walk(dst_path):
    for file in files:
        if file.endswith(('.mp4', '.avi', '.mov')):
            print(os.path.join(root, file))


✅ Dataset successfully copied to:
/Users/noel_personal/Desktop/badminton_dataset

📂 Video files found:

/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/008.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/020.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/034.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/035.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/021.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/009.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/037.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/023.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/022.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/036.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/032.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_drive/026.mp4
/Users/noel_personal/Desktop/badminton_dataset/backhand_

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "imagefolder", 
    data_dir="/Users/noel_personal/Desktop/badminton_dataset", 
    split="train", 
    extensions={"video": [".mp4"]}  # Optional: enforces video format
)

print(dataset)
dataset[0]  # preview


/Users/noel_personal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/noel_personal/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: __init__() got an unexpected keyword argument 'extensions'